# Profile fitting

One of the main purposes of $\mathrm{LiMe}$ is to fit profiles to the observed lines. This is done using the $\tt{.fit}$ functions. For example, let's use a [spectrum from the resources folder](https://github.com/Vital-Fernandez/lime/tree/master/examples/0_resources) to fit $H\beta$:


In [ ]:
from pathlib import Path
import lime

# Specify the data location the observations
data_folder = Path('../0_resources')
sloan_SHOC579 = data_folder/'spectra'/'sdss_dr18_0358-51818-0504.fits'
bands_df_file = data_folder/'bands'/'SHOC579_bands.txt'

# Create the observation object
spec = lime.Spectrum.from_file(sloan_SHOC579, instrument='sdss', redshift=0.0475)

# Fit a line from the default label list
spec.fit.bands('H1_4861A')

And to visualize the fitted profile you can run:

In [ ]:
# Plot the line region
spec.plot.bands()

The [measurements](../4_references/outputs1_measurements.rst) are stored into the ``Spectrum.frame`` attributes as a pandas dataframe:

In [ ]:
spec.frame[['profile_flux', 'profile_flux_err', 'amp', 'center', 'sigma']]

## Profile and shape:

You can use the [$\mathrm{LiMe}$ notation](2_line_labels.ipynb#shape-suffix-(s)) to adjust the profile and/or shape of your profile. For example:

In [ ]:
# Lorentz profile fitting
spec.fit.bands('H1_4861A_p-l')
spec.plot.bands()

Or you can use the `shape` and `profile` arguments to set these conditions:

In [ ]:
# Lorentz profile fitting
spec.fit.bands('O3_4959A', profile='l')
spec.plot.bands()

<div class="alert alert-info">

**Please note:** The default shape and profile values in $\mathrm{LiMe}$ measurements are emission (**emi**) and Gaussian (**g**). The priority for deciding the final value of these parameters is: **label suffix** > **configuration file** > **lines table** > `profile/shape` **arguments in the *.fit* functions** > **default values**. To change the default values, check the [lines database configuration](../2_guides/0_lines_database.ipynb). 


</div>

## Multi-profile fitting

By default $\mathrm{LiMe}$ assumes a single component, however, in many cases this is not enough:

In [ ]:
# Fit a line from the default label list
spec.fit.bands('H1_6563A')
spec.plot.bands()

In thi case we, need to fit multiple profiles simultaneously. Let's start by making a wider band for $H\alpha$ and $[NII]6548,6583\mathring{A}$ doublet

In [ ]:
band_df = spec.retrieve.lines_frame(line_list=['H1_6563A'], band_vsigma=350)
band_df

If we repeat the fitting:

In [ ]:
spec.fit.bands('H1_6563A', bands=band_df)
spec.plot.bands()

Now the central band covers the three transitions. However, only one line was fitted. 

<div class="alert alert-success">

To fit the three components, we need to use the blended **(_b)** suffix and specify the three components in the fitting configuration

</div>



In [ ]:
fit_cfg = {'H1_6563A_b': 'H1_6563A+N2_6583A+N2_6548A'}
spec.fit.bands('H1_6563A_b', band_df, fit_cfg)
spec.plot.bands()

This is a better result, now in the measurements frame we have:

In [ ]:
spec.frame.loc[['H1_6563A', 'N2_6583A', 'N2_6548A'], ['intg_flux', 'profile_flux']]

<div class="alert alert-warning">

**Please remember:** In de-blended lines, all components share the same integrated flux and uncertainty (``intg_flux`` and ``intg_flux_err``), which are only affected by the pixel values within the central line band. The profile fluxes (``profile_flux`` and ``profile_flux_err``) depend on the fitted profiles of each component.


</div>

## Constrain the profile parameters

In order to adjust the fittings, you can constrain the parameters of the profiles considered. For example, for a Gaussian profile, if the mathematical expression is:

$\mathrm{F_{\lambda}=\sum_{i}A_{i}e^{-\left(\frac{\lambda-\mu_{i}}{2\sigma_{i}}\right)^{2}} + m{\lambda} + c}$

You can introduce in the $\mathrm{LiMe}$ configuration files the line label followed by the profile name. For example, in the previous fitting:
 * $\tt{H1\_6563A\_amp}$: the line amplitude, i.e., the height of the Gaussian profile above the continuum level $(A_i)$. 
 * $\tt{H1\_6563A\_center}$: the observed Gaussian centroid in the spectrum wavelength units $(\mu_i)$.
 * $\tt{H1\_6563A\_sigma}$: the Gaussian profile sigma width in the spectrum wavelength units $(\sigma_i)$.
 * $\tt{H1\_6563A\_cont\_slope}$: the local continuum gradient. In blended lines, there is only one continuum slope labeled after the first component $(m)$.
 * $\tt{H1\_6563A\_cont\_intercept}$: the linear flux at the blue edge of the central band. In blended lines, there is still only one continuum intercept labeled after the first component $(c)$.


<div class="alert alert-success">

**Please remember:** Any value for the ``_center`` wavelength must be given in the rest frame; $\mathrm{LiMe}$ will apply the redshift correction. Similarly, the ``_amp`` values must use the same units as the input flux, and $\mathrm{LiMe}$ will apply the normalization.

</div>

LiMe transforms each of these entries into [an LmFIT parameter](https://lmfit.github.io/lmfit-py/parameters.html), which can have the following arguments:

* ``value``: Initial value of the parameter. $\mathrm{LiMe}$ provides an initial guess for the parameters from the [integrated measurements](https://lime-stable.readthedocs.io/en/latest/documentation/measurements.html#integrated-properties).
* ``vary``: Whether the parameter is free during fitting (default is True). If set to *False*, the initial ``value`` will remain unchanged.
* ``min``: Lower bound for the parameter value. The default value is -numpy.inf (no lower bound).
* ``max``: Upper bound for the parameter value. The default value is numpy.inf (no upper bound).
* ``expr``: Mathematical expression to constrain the value during fitting. The default value is None.

Let's try some of these parameter constrains:

In [ ]:
line = 'H1_6563A_b'
fit_conf = {'H1_6563A_b': 'H1_6563A+H1_6563A_k-1+N2_6584A+N2_6548A',        # Line components of the line
            'N2_6548A_amp': {'expr': 'N2_6584A_amp/2.94'},                  # [NII] amplitude constrained by the emissivity ratio
            'N2_6548A_kinem': 'N2_6584A',                                   # Tie the kinematics of the [NII] doublet
            'H1_6563A_k-1_center': {'value':6562, 'min': 6561, 'max':6563}, # Range for the wide Hα value
            'H1_6563A_k-1_sigma': {'expr':'>1.0*H1_6563A_sigma'}}           # Second Hα sigma must be higher than first sigma

# Second attempt including the fit configuration
spec.fit.bands(line, band_df, fit_conf)
spec.plot.bands()

In this fitting, we did several improvements:

* The amplitude ratio of the $[NII]6548,6584\mathring{A}$ is equal to the theoretical relation (2.94). This was done by using the ``'expr': 'N2_6584A_amp/2.94'`` to force the ``N2_6548A_amp`` to adhere to this relation.

In [ ]:
spec.frame.loc['N2_6584A', 'amp']/spec.frame.loc['N2_6548A', 'amp']

 * The nitrogen doublet has the same kinematics by using the ``N2_6548A_kinem': 'N2_6584A'``

In [ ]:
spec.frame.loc[['N2_6584A', 'N2_6548A'], ['v_r', 'sigma_vel']]

 * We added a kinematic component to $H\alpha$ (``H1_6563A_k-1``). The sigma of this component is wider thanks to the ``'H1_6563A_k-1_sigma': {'expr':'>1.0*H1_6563A_sigma'}`` term

In [ ]:
spec.frame.loc[['H1_6563A', 'H1_6563A_k-1'], 'sigma']

 * The sigma of the wide component has a user innitial value which is constrained to 1 angstrom via the ``'H1_6563A_k-1_center': {'value':6562, 'min': 6561, 'max':6563}`` term.

You are invited to try different techniques 

## Failed fittings

Depending on the complexity of the fitting, the minimizer can fail to converge. In this case the profile lines will be displayed in red:

In [ ]:
line = 'H1_6563A_b'
fit_conf = {'H1_6563A_b': 'H1_6563A+H1_6563A_k-1+N2_6584A+N2_6548A'} 
spec.fit.bands(line, band_df, fit_conf)
spec.plot.bands()

And the uncertainty of the profile components will be ``np.nan``:

In [ ]:
spec.frame.loc[['H1_6563A', 'H1_6563A_k-1'], ['profile_flux', 'profile_flux_err', 'amp', 'amp_err', 'center', 'center_err', 'sigma', 'sigma_err']]

In order to fix a failed fitting, you can add more constrains on the parameters. In order to get a better insight on the causes behind the failed fitting you can access the [LmFIT report](https://lmfit.github.io/lmfit-py/model.html#lmfit.model.ModelResult.fit_report) via the ``.fit.report()``:

In [ ]:
spec.fit.report()

We can see in this report that there is a big change between the initial center for the Halpha components:

```
H1_6563A_center:  16599.3998 (init = 6874.448)
H1_6563A_k-1_center: -2850.16426 (init = 6874.448)
```

This means that you should constrain these parameters more.

<div class="alert alert-info">

**Please remember:** Once we start to fit profiles with more than 10 dimensions (four Gaussian components) tranditional minimizers are very sensitive to the boundary conditions and cannot explore the parameter space efficiently. Future $\mathrm{LiMe}$ upgrades will explore more advance samplers to automatically decide the number of components and estimate the parameter values distributions.
</div>

## Read fitting configuration from a text file

In most cases, you will be interested in fitting multiple lines from multiple spectra. In order to simplify the configuration entries typing, it is recomended that you put your configuration into a text file with a [.toml configuration format](https://toml.io/en/) and use that as an input for your ``.fit`` functions.

For example, lets save the previous configuration to a text file:

In [ ]:
fit_cfg_str = """[default_line_fitting]
H1_6563A_b = 'H1_6563A+H1_6563A_k-1+N2_6584A+N2_6548A'
N2_6548A_amp = 'expr:N2_6584A_amp/2.94'
N2_6548A_kinem = 'N2_6584A'
H1_6563A_k-1_center = 'value:6562,min:6561,max:6563'
H1_6563A_k-1_sigma = 'expr:>1.0*H1_6563A_sigma'
"""

# Save to file while keeping line breaks
with open("../0_resources/Halpha_cfg.toml", "w", encoding="utf-8") as file:
    file.write(fit_cfg_str)

This produces a '.toml' file which can be read using $\tt{lime.load\_cfg}$ function:

In [ ]:
fit_cfg = lime.load_cfg("../0_resources/Halpha_cfg.toml")
fit_cfg

When the $\tt{lime.load\_cfg}$ function finds a ``[section]`` with the ``_line_fitting`` it will format the entries to match $\mathrm{LiMe}$ expected ittin format. This way you don't need to type the **"{"** and **"}"** characters (but you must avoid white spaces and put all the item values between **' '**). 

This dictionary can be introduced to the fitting function directly:

In [ ]:
line = 'H1_6563A_b'
spec.fit.bands(line, band_df, fit_cfg)
spec.plot.bands()

Or we can directly input the text file address:

In [ ]:
line = 'H1_6563A_b'
spec.fit.bands(line, band_df, "Halpha_cfg.toml")
spec.plot.bands()

## Save the results

$\mathrm{LiMe}$ can save the results into different formats using the $\tt{lime.Spectrum.save\_frame}$ function:

In [ ]:
# Save to a text file
spec.save_frame('../0_resources/example1_linelog.txt')

Additional file formats allow the user to save the current data to a specific page:

In [ ]:
spec.save_frame('../0_resources/example1_linelog.fits', page='SHOC579')
spec.save_frame('../0_resources/example1_linelog.xlsx', page='SHOC579')

Finally, if you have latex and pylatex installed on your system you can also save the results as a pdf file:

In [ ]:
spec.save_frame('../0_resources/example1_linelog.pdf', param_list=['eqw', 'profile_flux', 'profile_flux_err'])

The argument ``param_list`` constrains the output measurements to the input list.

## Takeaways

* By default $\mathrm{LiMe}$ will fit a Gaussian profile to a line.
* To fit multiple components to a line you need to add the blended suffix to the line label (for example ``H1_6563A_b``) and its components into the fitting configuration: (for example ``H1_6563A_b='H1_6563A+N2_6585A+N2_6548A'``)
* You can constrain the components profile parameter using the value,min,max,expr,vary attributes to the configuration (for example ``'expr:>1.0*H1_6563A_sigma'``)
* In very complex profiles (4 components or more) the minimizer is very sensitive to the input boundary conditions a lot of tinkering might be necessary in the current algorithm version to succesfully fit these profiles.
* It is recomended you save your profile settings in a .toml file and input them on the fitting functions. As long as the corresponding ``[section]`` has the ``_line_fitting`` suffix the entries will be formated to the expected format. If no section name is introduced $\mathrm{LiMe}$ will use the data from the ``[default_line_fitting]``